# clone the repositories in their respective folders

In [1]:
!git clone https://github.com/Rudrabha/Wav2Lip.git Wav2Lip-master
!git clone https://github.com/TencentARC/GFPGAN.git GFPGAN-master

Cloning into 'Wav2Lip-master'...
remote: Enumerating objects: 369, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 369 (delta 4), reused 2 (delta 1), pack-reused 360
Receiving objects: 100% (369/369), 528.94 KiB | 4.20 MiB/s, done.
Resolving deltas: 100% (202/202), done.
Cloning into 'GFPGAN-master'...
remote: Enumerating objects: 523, done.
remote: Counting objects: 100% (523/523), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 523 (delta 279), reused 466 (delta 264), pack-reused 0
Receiving objects: 100% (523/523), 5.37 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (279/279), done.


# install the necessary dependencies.

In [2]:
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
gfpganPath = '/content/GFPGAN-master'
wav2lipPath = '/content/Wav2Lip-master'

!wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'

!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O {wav2lipPath}'/checkpoints/wav2lip_gan.pth'
#!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O {wav2lipPath}'/checkpoints/wav2lip.pth'

!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'

!pip install -r requirements.txt
!pip install -U librosa==0.8.1 # The process will fail without downgrading librosa
!mkdir inputs

!cd $gfpganFolderName && python setup.py develop
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'

%cd {basePath}

from IPython.display import clear_output
clear_output()

print("Installation complete.")

Installation complete.


# using wave2lip sync the audio and video.
Wav2Lip	Highly accurate lip-sync

Wav2Lip + GAN	Slightly inferior lip-sync, but better visual quality

In [7]:
#@markdown Adjust the parameters below then run the code block to synthesize the speech onto your video file:
basePath = '/content/'
import os
outputPath = basePath+'/outputs'
inputAudio = 'input_audio.mp3' #@param{type:"string"}
inputAudioPath = basePath + '/inputs/' + inputAudio
inputVideo = 'pic.png' #@param{type:"string"}
inputVideoPath = basePath + '/inputs/'+inputVideo
lipSyncedOutputPath = basePath + '/outputs/result.mp4'
model = "wav2lip_gan" #@param ["wav2lip", "wav2lip_gan"] {type:"string"}


if not os.path.exists(outputPath):
  os.makedirs(outputPath)

from IPython.display import clear_output
clear_output()

!cd $wav2lipFolderName && python inference.py \
--checkpoint_path checkpoints/{model}.pth \
--face {inputVideoPath} \
--audio {inputAudioPath} \
--outfile {lipSyncedOutputPath}

Using cuda for inference.
Number of frames available for inference: 1
Extracting raw audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enab

# Enhance the video with GFPGAN

In [8]:
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
  os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath, str(frameNumber).zfill(4)+'.jpg'), image)

!cd $gfpganFolderName && \
  python inference_gfpgan.py -i $unProcessedFramesFolderPath -o $outputPath -v 1.3 -s 2 --only_center_face --bg_upsampler None

import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

#Get FPS of original video for writer
inputVideoPath = outputPath+'/result.mp4'
vidcap = cv2.VideoCapture(inputVideoPath)
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("The video is "+str(fps)+" FPS.")

batch = 0
batchSize = 1300
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end, end="\r")
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
      print("Video video writer")
  out = cv2.VideoWriter(processedVideoOutputPath+'/output_'+str(batch).zfill(4)+'.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()
  print("Video upscaling complete.")

FPS:  25.0 Frames:  574


100%|██████████| 574/574 [00:08<00:00, 66.82it/s]


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Processing 0000.jpg ...
Processing 0001.jpg ...
Processing 0002.jpg ...
Processing 0003.jpg ...
Processing 0004.jpg ...
Proces

  0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 3/688 [00:00<00:23, 29.29it/s]

...

Video video writer
Video video writer
Video video writer



100%|██████████| 688/688 [00:27<00:00, 25.24it/s]

Video video writer



100%|██████████| 1/1 [00:39<00:00, 39.16s/it]

Video upscaling complete.


# put the audio on the enhance video.
after enhancing each frame one by one it lost its audio so lets just put the audio on the enhanced video.

In [6]:
from moviepy.editor import VideoFileClip, AudioFileClip

def add_audio_to_video(video_path, audio_path, output_path):
    # Load the video clip
    video_clip = VideoFileClip(video_path)

    # Load the audio clip
    audio_clip = AudioFileClip(audio_path)

    # Set the video clip's audio to the loaded audio clip
    video_clip = video_clip.set_audio(audio_clip)

    # Write the result to a file
    video_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

video_path = "/content/outputs/output_0000.mp4"
audio_path = "/content/inputs/input_audio.mp3"
output_path = "/content/outputs/output_video.mp4"

add_audio_to_video(video_path, audio_path, output_path)

Moviepy - Building video /content/outputs/output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/outputs/output_video.mp4



Moviepy - Done !
Moviepy - video ready /content/outputs/output_video.mp4
